# Generative Search with OctoAI

OctoAI offers a variety of embedding and generative models via their API. 

This recipe will show you how to generate embeddings and use generative models with OctoAI

## Requirements

1. Weaviate cluster
    1. You can create a 14-day free sandbox on [WCD](https://console.weaviate.cloud/)
    2. [Embedded Weaviate](https://weaviate.io/developers/weaviate/installation/embedded)
    3. [Local deployment](https://weaviate.io/developers/weaviate/installation/docker-compose#starter-docker-compose-file)
    4. [Other options](https://weaviate.io/developers/weaviate/installation)

2. OctoAI API key. Grabe on [here](https://octoai.cloud/).

In [2]:
import weaviate, os
from weaviate.embedded import EmbeddedOptions
import weaviate.classes as wvc
import weaviate.classes.config as wc
import requests, json
import weaviate.classes.query as wq

## Connect to Weaviate

Only choose one option from the below.

**Weaviate Cloud Deployment**

In [ ]:
WCD_URL = os.environ["WEAVIATE_URL"] # Replace with your Weaviate cluster URL
WCD_AUTH_KEY = os.environ["WEAVIATE_AUTH"] # Replace with your cluster auth key
OCTOAI_KEY = os.environ["OCTOAI_API_KEY"] # Replace with your OctoAI key

# Weaviate Cloud Deployment
client = weaviate.connect_to_wcs(
    cluster_url=WCD_URL,
    auth_credentials=weaviate.auth.AuthApiKey(WCD_AUTH_KEY),
      headers={ "X-OctoAI-Api-Key": OCTOAI_KEY}
)

print(client.is_ready())

**Embedded Weaviate**

In [ ]:
# OCTOAI_KEY = os.environ["OCTOAI_API_KEY"] # Replace with your OctoAI key


# client = weaviate.WeaviateClient(
#     embedded_options=EmbeddedOptions(
#         version="1.26.1",
#         additional_env_vars={
#             "ENABLE_MODULES": "text2vec-octoai, generative-octoai"
#         }),
#         additional_headers={
#             "X-OctoAI-Api-Key": OCTOAI_KEY
#         }
# )

# client.connect()

**Local Deployment**

In [ ]:
# OCTOAI_KEY = os.environ["OCTOAI_API_KEY"] # Replace with your OctoAI key

# client = weaviate.connect_to_local(
#   headers={
#     "X-OctoAI-Api-Key": OCTOAI_KEY
#   }
# )
# print(client.is_ready())

## Create a collection
Collection stores your data and vector embeddings.

Full list of [generative models](https://weaviate.io/developers/weaviate/model-providers/octoai/generative#available-models)

In [ ]:
# Note: in practice, you shouldn't rerun this cell, as it deletes your data
# in "JeopardyQuestion", and then you need to re-import it again.

# Delete the collection if it already exists
if (client.collections.exists("JeopardyQuestion")):
    client.collections.delete("JeopardyQuestion")

client.collections.create(
    name="JeopardyQuestion",

    vectorizer_config=wc.Configure.Vectorizer.text2vec_octoai( # specify the vectorizer and model
        model="thenlper/gte-large",
    ),

    generative_config=wc.Configure.Generative.octoai( # specify the generarive model
        model="meta-llama-3-70b-instruct"
    )

    properties=[ # defining properties (data schema) is optional
        wc.Property(name="Question", data_type=wc.DataType.TEXT), 
        wc.Property(name="Answer", data_type=wc.DataType.TEXT),
        wc.Property(name="Category", data_type=wc.DataType.TEXT, skip_vectorization=True), 
    ]
)

print("Successfully created collection: JeopardyQuestion.")

## Import the Data

In [ ]:
url = 'https://raw.githubusercontent.com/weaviate/weaviate-examples/main/jeopardy_small_dataset/jeopardy_tiny.json'
resp = requests.get(url)
data = json.loads(resp.text)

# Get a collection object for "JeopardyQuestion"
jeopardy = client.collections.get("JeopardyQuestion")

# Insert data objects
response = jeopardy.data.insert_many(data)

# Note, the `data` array contains 10 objects, which is great to call insert_many with.
# However, if you have a milion objects to insert, then you should spit them into smaller batches (i.e. 100-1000 per insert)

if (response.has_errors):
    print(response.errors)
else:
    print("Insert complete.")

## Generative Search Queries

### Single Result

Single Result makes a generation for each individual search result. 

In the below example, I want to create a Facebook ad from the Jeopardy question about Elephants. 

In [ ]:
generatePrompt = "Turn the following Jeogrady question into a Facebook Ad: {question}"

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.generate.near_text(
    query="Elephants",
    limit=2,
    single_prompt=generatePrompt
)

for item in response.objects:
    print(json.dumps(item.properties, indent=1))
    print("-----vvvvvv-----")
    print(item.generated)
    print("-----^^^^^^-----")

### Grouped Result

Grouped Result generates a single response from all the search results. 

The below example is creating a Facebook ad from the 2 retrieved Jeoprady questions about animals. 

In [ ]:
generateTask = "Explain why these Jeopardy questions are under the Animals category."

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.generate.near_text(
    query="Animals",
    limit=3,
    grouped_task=generateTask
)

print(response.generated)